# 4-CRU Mechanism Kinematic Calculation Handbook

In [1]:
import numpy as np
import numpy.matlib
import math
from sympy import * # symbolic calculation for IK
from tf import transformations as tfs 

## Notes on [Transformation.py](https://github.com/ROAR-Lab-Spine/geometry/blob/melodic-devel/tf/src/tf/transformations.py)


1. Quaternions $q = ix+jy+kz+w$ are represented as $[x, y, z, w ]$= $[x_1, x_2, x_3, x_0]$.

## Notes on [SymPy’s documentation](https://docs.sympy.org/latest/index.html)

1. [Matrix Functions](https://docs.sympy.org/latest/modules/matrices/common.html?highlight=transpose#sympy.matrices.common.MatrixCommon.transpose)
2. [Matrix Operations](https://docs.sympy.org/latest/modules/matrices/matrices.html?highlight=matrix%20operation)
3. [Polynomial and Groebner bases¶](https://docs.sympy.org/latest/modules/polys/basics.html?highlight=polynomial%20solve)
4. Primary Decomposition

## Notes on SINGULAR
1. [Related Publications](https://www.singular.uni-kl.de/index.php/publications/singular-related-publications.html)

In [2]:
tf_init = tfs.identity_matrix()
# print tf_init
des_quat = tfs.quaternion_about_axis(0, (0, 0, 1))
des_xyz = np.array([0,0,0.1])
print des_quat
print des_xyz

[0. 0. 0. 1.]
[0.  0.  0.1]


In [3]:
# Set up Symbolic variables
X, Y, Z = symbols('X, Y, Z')
x_0, x_1, x_2, x_3 = symbols('x_0, x_1, x_2, x_3')
a, b, c, d = symbols('a, b, c, d')
h_1, h_2, h_3, h_4 = symbols('h_1, h_2, h_3, h_4')
r = symbols('r')

In [4]:
# Position of the C joint with respect to the base frame
r0_B1 = Matrix([a, -b, h_1, 1])
r1_C1 = Matrix([c, -d, 0, 1])

r0_B2 = Matrix([a, b, h_2, 1])
r1_C2 = Matrix([c, d, 0, 1])

r0_B3 = Matrix([-a, b, h_3, 1])
r1_C3 = Matrix([-c, d, 0, 1])

r0_B4 = Matrix([-a, -b, h_4, 1])
r1_C4 = Matrix([-c, -d, 0, 1])

In [5]:
R0_1 = Matrix([[x_0**2 + x_1**2 - x_2**2 - x_3**2, 2.0*(x_1*x_2 - x_0*x_3), 2.0*(x_0*x_2 + x_1*x_3)],
                [2.0*(x_0*x_3 + x_1*x_2),x_0**2 - x_1**2 + x_2**2 - x_3**2,2.0*(x_2*x_3 - x_0*x_1)],
                [2.0*(x_1*x_3 - x_0*x_2),2.0*(x_0*x_1 + x_2*x_3),x_0**2 - x_1**2 - x_2**2 + x_3**2]])
r0_1 = Matrix([X, Y, Z, 1.0])  
T0_1 = R0_1.col_join(zeros(1,3))
T0_1 = T0_1.row_join(r0_1)
T0_1

Matrix([
[x_0**2 + x_1**2 - x_2**2 - x_3**2,        -2.0*x_0*x_3 + 2.0*x_1*x_2,         2.0*x_0*x_2 + 2.0*x_1*x_3,   X],
[        2.0*x_0*x_3 + 2.0*x_1*x_2, x_0**2 - x_1**2 + x_2**2 - x_3**2,        -2.0*x_0*x_1 + 2.0*x_2*x_3,   Y],
[       -2.0*x_0*x_2 + 2.0*x_1*x_3,         2.0*x_0*x_1 + 2.0*x_2*x_3, x_0**2 - x_1**2 - x_2**2 + x_3**2,   Z],
[                                0,                                 0,                                 0, 1.0]])

In [6]:
# Vector along the U-U Rods
u0_1 = T0_1*r1_C1 - r0_B1
u0_2 = T0_1*r1_C2 - r0_B2
u0_3 = T0_1*r1_C3 - r0_B3
u0_4 = T0_1*r1_C4 - r0_B4

In [7]:
r0_ee_to_C2 = T0_1*r1_C2
r0_ee_to_C2 = r0_ee_to_C2.subs([(X, 0),(Y, 0),(Z, 0)])

In [8]:
# Scalar Triple Product 
f_5 = simplify(expand(u0_1.dot(u0_1))) - r**2
f_6 = simplify(expand(u0_2.dot(u0_2))) - r**2
f_7 = simplify(expand(u0_3.dot(u0_3))) - r**2
f_8 = simplify(expand(u0_4.dot(u0_4))) - r**2
f_9 = x_0**2+x_1**2+x_2**2+x_3**2-1.0

# Substitute Additional Mode Geometric Condition A
f_5A = f_5.subs([(c, a),(x_2, 0),(x_3, 0),(X,0)],simultaneous=True)
f_6A = f_6.subs([(c, a),(x_2, 0),(x_3, 0),(X,0)],simultaneous=True)
f_7A = f_7.subs([(c, a),(x_2, 0),(x_3, 0),(X,0)],simultaneous=True)
f_8A = f_8.subs([(c, a),(x_2, 0),(x_3, 0),(X,0)],simultaneous=True)
f_9A = f_9.subs([(c, a),(x_2, 0),(x_3, 0),(X,0)],simultaneous=True)

In [9]:
f_9A_sq = f_9A*f_9A
f_9A_sq_expd = expand(f_9A_sq)
f_9A_sq

(x_0**2 + x_1**2 - 1.0)**2

In [10]:
f_9A_sq_expd.subs(-2.0*x_0**2-2.0*x_1**2, -2)

x_0**4 + 2*x_0**2*x_1**2 - 2.0*x_0**2 + x_1**4 - 2.0*x_1**2 + 1.0

In [11]:
f_9A_sq_expd

x_0**4 + 2*x_0**2*x_1**2 - 2.0*x_0**2 + x_1**4 - 2.0*x_1**2 + 1.0

In [12]:
f_5A = collect(f_5A, (X,Y,Z,r,a,b,c,d))
f_5A

Y**2 + Y*(2*b - 2*d*x_0**2 + 2*d*x_1**2) + Z**2 + Z*(-4.0*d*x_0*x_1 - 2*h_1) + a**2*(x_0**4 + 2*x_0**2*x_1**2 - 2*x_0**2 + x_1**4 - 2*x_1**2 + 1) + b**2 + b*(-2*d*x_0**2 + 2*d*x_1**2) + d**2*(x_0**4 + 2.0*x_0**2*x_1**2 + x_1**4) + 4.0*d*h_1*x_0*x_1 + h_1**2 - r**2

In [13]:
# eliminate the quaternion identity
f_5A = f_5A.subs(x_0**4+2.0*x_0**2*x_1**2+x_1**4, 1)
f_5A = f_5A.subs(f_9A_sq_expd-1.0,-1.0)
f_5A

Y**2 + Y*(2*b - 2*d*x_0**2 + 2*d*x_1**2) + Z**2 + Z*(-4.0*d*x_0*x_1 - 2*h_1) + b**2 + b*(-2*d*x_0**2 + 2*d*x_1**2) + d**2 + 4.0*d*h_1*x_0*x_1 + h_1**2 - r**2

In [14]:
f_6A = collect(f_6A, (X,Y,Z,r,a,b,c,d))
f_6A = f_6A.subs(x_0**4+2.0*x_0**2*x_1**2+x_1**4, 1)
f_6A = f_6A.subs(f_9A_sq_expd-1.0,-1.0)
f_6A

Y**2 + Y*(-2*b + 2*d*x_0**2 - 2*d*x_1**2) + Z**2 + Z*(4.0*d*x_0*x_1 - 2*h_2) + b**2 + b*(-2*d*x_0**2 + 2*d*x_1**2) + d**2 - 4.0*d*h_2*x_0*x_1 + h_2**2 - r**2

In [15]:
f_7A = collect(f_7A, (X,Y,Z,r,a,b,c,d))
f_7A = f_7A.subs(x_0**4+2.0*x_0**2*x_1**2+x_1**4, 1)
f_7A = f_7A.subs(f_9A_sq_expd-1.0,-1.0)
f_7A

Y**2 + Y*(-2*b + 2*d*x_0**2 - 2*d*x_1**2) + Z**2 + Z*(4.0*d*x_0*x_1 - 2*h_3) + b**2 + b*(-2*d*x_0**2 + 2*d*x_1**2) + d**2 - 4.0*d*h_3*x_0*x_1 + h_3**2 - r**2

In [16]:
f_8A = collect(f_8A, (X,Y,Z,r,a,b,c,d))
f_8A = f_8A.subs(x_0**4+2.0*x_0**2*x_1**2+x_1**4, 1)
f_8A = f_8A.subs(f_9A_sq_expd-1.0,-1.0)
f_7A

Y**2 + Y*(-2*b + 2*d*x_0**2 - 2*d*x_1**2) + Z**2 + Z*(4.0*d*x_0*x_1 - 2*h_3) + b**2 + b*(-2*d*x_0**2 + 2*d*x_1**2) + d**2 - 4.0*d*h_3*x_0*x_1 + h_3**2 - r**2

# Inverse Kinematics (Schoenfiles Mode: Geometric A)

In [17]:
# Schonflies Mode
# T0_1_H1 = T0_1.subs([(x_1, 0),(x_2, 0)],simultaneous=True)
des_pose = tfs.identity_matrix()

f_5_H1 = f_5.subs([(X, des_xyz[0]),
                   (Y, des_xyz[1]),
                   (Z, des_xyz[2]),
                   (x_1, des_quat[0]),
                   (x_2, des_quat[1]),
                   (x_3, des_quat[2]),
                   (x_0, des_quat[3])],
                  simultaneous=True) 
h_1_sols = solve(f_5_H1, h_1)

In [18]:
h_2_sols = solve(f_6, h_2)
h_2_sols[1].subs([(x_1, 0),(x_2,0)])
# We can change the signs of a,b,c,d corresponding to the 

Z + sqrt(-X**2 + 2.0*X*a - 2.0*X*c*x_0**2 + 2.0*X*c*x_3**2 + 4.0*X*d*x_0*x_3 - Y**2 + 2.0*Y*b - 4.0*Y*c*x_0*x_3 - 2.0*Y*d*x_0**2 + 2.0*Y*d*x_3**2 - a**2 + 2.0*a*c*x_0**2 - 2.0*a*c*x_3**2 - 4.0*a*d*x_0*x_3 - b**2 + 4.0*b*c*x_0*x_3 + 2.0*b*d*x_0**2 - 2.0*b*d*x_3**2 - c**2*x_0**4 - 2.0*c**2*x_0**2*x_3**2 - c**2*x_3**4 - d**2*x_0**4 - 2.0*d**2*x_0**2*x_3**2 - d**2*x_3**4 + r**2)

# Inverse Kinematics (Additional Mode: Geometric A)

In [19]:
# Additional Mode (Gemetric A)
g_1A = factor(f_5A - f_8A)
g_1A

4.0*(1.0*h_1 - 1.0*h_4)*(-0.5*Z + 1.0*d*x_0*x_1 + 0.25*h_1 + 0.25*h_4)

In [20]:
g_2A = factor(f_5A + f_8A)
g_2A

-8.0*(-0.25*Y**2 - 0.5*Y*b + 0.5*Y*d*x_0**2 - 0.5*Y*d*x_1**2 - 0.25*Z**2 + 1.0*Z*d*x_0*x_1 + 0.25*Z*h_1 + 0.25*Z*h_4 - 0.25*b**2 + 0.5*b*d*x_0**2 - 0.5*b*d*x_1**2 - 0.25*d**2 - 0.5*d*h_1*x_0*x_1 - 0.5*d*h_4*x_0*x_1 - 0.125*h_1**2 - 0.125*h_4**2 + 0.25*r**2)

In [21]:
g_3A = factor(f_6A - f_7A)
g_3A = g_3A .args[1]*(g_3A.args[0]*g_3A.args[2])
g_3A

(1.0*h_2 - 1.0*h_3)*(-2.0*Z - 4.0*d*x_0*x_1 + 1.0*h_2 + 1.0*h_3)

In [22]:
g_4A = expand(factor(f_6A + f_7A))
g_4A

2.0*Y**2 - 4.0*Y*b + 4.0*Y*d*x_0**2 - 4.0*Y*d*x_1**2 + 2.0*Z**2 + 8.0*Z*d*x_0*x_1 - 2.0*Z*h_2 - 2.0*Z*h_3 + 2.0*b**2 - 4.0*b*d*x_0**2 + 4.0*b*d*x_1**2 + 2.0*d**2 - 4.0*d*h_2*x_0*x_1 - 4.0*d*h_3*x_0*x_1 + 1.0*h_2**2 + 1.0*h_3**2 - 2.0*r**2

In [23]:
# Reduce to 3 Equations 
e_1A = expand(factor(g_2A.subs(h_1, h_4)))/2
e_1A

Y**2 + 2.0*Y*b - 2.0*Y*d*x_0**2 + 2.0*Y*d*x_1**2 + Z**2 - 4.0*Z*d*x_0*x_1 - 2.0*Z*h_4 + b**2 - 2.0*b*d*x_0**2 + 2.0*b*d*x_1**2 + d**2 + 4.0*d*h_4*x_0*x_1 + h_4**2 - 1.0*r**2

In [24]:
#***Choose the factor that does not lead to actuation configuration
e_2A = g_3A.args[1]
e_2A

-2.0*Z - 4.0*d*x_0*x_1 + 1.0*h_2 + 1.0*h_3

In [25]:
e_3A = g_4A
e_3A
factor(e_3A)

-8.0*(-0.25*Y**2 + 0.5*Y*b - 0.5*Y*d*x_0**2 + 0.5*Y*d*x_1**2 - 0.25*Z**2 - 1.0*Z*d*x_0*x_1 + 0.25*Z*h_2 + 0.25*Z*h_3 - 0.25*b**2 + 0.5*b*d*x_0**2 - 0.5*b*d*x_1**2 - 0.25*d**2 + 0.5*d*h_2*x_0*x_1 + 0.5*d*h_3*x_0*x_1 - 0.125*h_2**2 - 0.125*h_3**2 + 0.25*r**2)

In [65]:
# solve for h_4 (=h_1), h_2, and h_3

# invs_kin_sols_geo_A = solve((e_1A, e_2A, e_3A), (h_2, h_3, h_4))
# invs_kin_sols_geo_A

h_1_h_4_sol = simplify(solve(e_1A, h_4))
h_1_h_4_sol[0]

Z - 2.0*d*x_0*x_1 - sqrt(-Y**2 - 2.0*Y*b + 2.0*Y*d*x_0**2 - 2.0*Y*d*x_1**2 - b**2 + 2.0*b*d*x_0**2 - 2.0*b*d*x_1**2 + 4.0*d**2*x_0**2*x_1**2 - d**2 + r**2)

In [57]:
# https://stackoverflow.com/questions/53092738/eliminate-a-variable-to-relate-two-functions-in-python-using-sympy
# https://github.com/sympy/sympy/issues/2720
d_1A = factor(e_3A.subs(h_2, solve(e_2A, h_2)[0]))
h_3_sol = solve((d_1A,f_9A), h_3)

In [64]:
h_3_sol[1][0]

Z + 2.0*d*x_0*x_1 + sqrt(-Y**2 + 2.0*Y*b - 2.0*Y*d*x_0**2 + 2.0*Y*d*x_1**2 - b**2 + 2.0*b*d*x_0**2 - 2.0*b*d*x_1**2 + 4.0*d**2*x_0**2*x_1**2 - d**2 + r**2)

In [63]:
h_2_sol = (solve(e_2A, h_2)[0]).subs(h_3, h_3_sol[1][0])
h_2_sol

1.0*Z + 2.0*d*x_0*x_1 - sqrt(-Y**2 + 2.0*Y*b - 2.0*Y*d*x_0**2 + 2.0*Y*d*x_1**2 - b**2 + 2.0*b*d*x_0**2 - 2.0*b*d*x_1**2 + 4.0*d**2*x_0**2*x_1**2 - d**2 + r**2)